In [1]:
%reload_ext autoreload
%autoreload 2
import os,sys
import pickle
import pandas as pd
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import openslide
import skimage.io
import PIL
from IPython.display import Image, display
from pylab import rcParams
from PIL import Image

from input.inputPipeline_stiching import PandaPatchDataset

In [2]:
filename = "Resnext50_medreso_36patch_adam_cosine_bin_0_cfm_name.pkl"
f = open(filename,'rb')
cfm_name = pickle.load(f)
f.close()

In [3]:
discription = []
no_slides = []
for i in range(6):
    for j in range(6):
        if i != j:
            discription.append("{} slides are labeled as {} but predicts as {}.\n".format(len(cfm_name['{}_{}'.format(i, j)]),
                                                                            i, j))
            no_slides.append(len(cfm_name['{}_{}'.format(i, j)]))

In [4]:
sort_idx = sorted(range(len(no_slides)), key=lambda k: no_slides[k])[::-1]

In [6]:
for i,idx in enumerate(sort_idx):
    print(discription[idx])
#     if i >= 10:
#         break

104 slides are labeled as 1 but predicts as 2.

95 slides are labeled as 5 but predicts as 4.

93 slides are labeled as 2 but predicts as 1.

74 slides are labeled as 4 but predicts as 3.

71 slides are labeled as 2 but predicts as 3.

70 slides are labeled as 3 but predicts as 4.

70 slides are labeled as 0 but predicts as 1.

45 slides are labeled as 1 but predicts as 0.

42 slides are labeled as 3 but predicts as 2.

36 slides are labeled as 4 but predicts as 5.

32 slides are labeled as 5 but predicts as 3.

22 slides are labeled as 4 but predicts as 2.

16 slides are labeled as 1 but predicts as 3.

14 slides are labeled as 0 but predicts as 2.

10 slides are labeled as 4 but predicts as 1.

10 slides are labeled as 3 but predicts as 5.

10 slides are labeled as 2 but predicts as 4.

9 slides are labeled as 3 but predicts as 0.

8 slides are labeled as 4 but predicts as 0.

7 slides are labeled as 5 but predicts as 2.

7 slides are labeled as 3 but predicts as 1.

4 slides are lab

### Visualize the wrong slides

In [9]:
# def overlay_mask_on_slide(images, center='radboud', alpha=0.8, max_size=(800, 800), preds = 0):
#     """Show a mask overlayed on a slide."""
#     N = len(images)
#     f, ax = plt.subplots(N//3,3, figsize=(18,22))

#     for i, image_id in enumerate(images):
#         if i > N // 3 * 3:
#             break
#         slide = openslide.OpenSlide(os.path.join(data_dir, f'{image_id}.tiff'))
#         mask = openslide.OpenSlide(os.path.join(mask_dir, f'{image_id}_mask.tiff'))
#         slide_data = slide.read_region((0,0), slide.level_count - 1, slide.level_dimensions[-1])
#         mask_data = mask.read_region((0,0), mask.level_count - 1, mask.level_dimensions[-1])
#         mask_data = mask_data.split()[0]
        
        
#         # Create alpha mask
#         alpha_int = int(round(255*alpha))
#         if center == 'radboud':
#             alpha_content = np.less(mask_data.split()[0], 2).astype('uint8') * alpha_int + (255 - alpha_int)
#         elif center == 'karolinska':
#             alpha_content = np.less(mask_data.split()[0], 1).astype('uint8') * alpha_int + (255 - alpha_int)

#         alpha_content = PIL.Image.fromarray(alpha_content)
#         preview_palette = np.zeros(shape=768, dtype=int)

#         if center == 'radboud':
#             # Mapping: {0: background, 1: stroma, 2: benign epithelium, 3: Gleason 3, 4: Gleason 4, 5: Gleason 5}
#             preview_palette[0:18] = (np.array([0, 0, 0, 0.5, 0.5, 0.5, 0, 1, 0, 1, 1, 0.7, 1, 0.5, 0, 1, 0, 0]) * 255).astype(int)
#         elif center == 'karolinska':
#             # Mapping: {0: background, 1: benign, 2: cancer}
#             preview_palette[0:9] = (np.array([0, 0, 0, 0, 1, 0, 1, 0, 0]) * 255).astype(int)

#         mask_data.putpalette(data=preview_palette.tolist())
#         mask_rgb = mask_data.convert(mode='RGB')
#         overlayed_image = PIL.Image.composite(image1=slide_data, image2=mask_rgb, mask=alpha_content)
#         overlayed_image.thumbnail(size=max_size, resample=0)
        
#         if N // 3 > 1:
#             ax[i//3, i%3].imshow(overlayed_image) 
#             ax[i//3, i%3].axis('off')
#         else:
#             ax[i%3].imshow(overlayed_image) 
#             ax[i%3].axis('off')
#         slide.close()
#         mask.close()       
#         data_provider = train.loc[image_id, 'data_provider']
#         isup_grade = train.loc[image_id, 'isup_grade']
#         gleason_score = train.loc[image_id, 'gleason_score']
        
#         if N // 3 > 1:
#             ax[i//3, i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")
#         else:
#             ax[i%3].set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score}")
#     f.savefig(f"./wrongPreds/label_{isup_grade}_pred_{preds}_w_mask.png")

In [11]:
# Location of the training images
BASE_PATH = './input/prostate-cancer-grade-assessment'

# image and mask directories
data_dir = f'{BASE_PATH}/train_images'
mask_dir = f'{BASE_PATH}/train_label_masks'

# Location of training labels
train = pd.read_csv('./input/csv_pkl_files/4_fold_whole_train.csv').set_index('image_id')

In [79]:
isup = 1
preds = 2
names = cfm_name['{}_{}'.format(isup, preds)]
# overlay_mask_on_slide(names[:9], preds = preds)

In [29]:
for img_id in names[:5]:
    im = skimage.io.MultiImage(os.path.join(data_dir, img_id + '.tiff'))[0]
    gls = train.loc[img_id, "gleason_score"]
    isup = train.loc[img_id, "isup_grade"]
    data_provider = train.loc[img_id, "data_provider"]
    try: 
        os.mkdir(f"./input/wrongPreds/label_{isup}_preds_{preds}/")
    except:
        pass
    skimage.io.imsave(f"./input/wrongPreds/label_{isup}_preds_{preds}/{isup}_{gls}_{data_provider}_{img_id}.png", im)

In [31]:
sz = 256
tsfm = None
image_dir = './input/panda-36x256x256-tiles-data-opt/train'
csv_file = './input/csv_pkl_files/4_fold_whole_train.csv'
dataset = PandaPatchDataset(csv_file, image_dir, sz, transform=tsfm, N = 36, rand = False)

In [54]:
def open_image(fn, convert_mode='RGB', after_open=None):
#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore", UserWarning)  # EXIF warning from TiffPlugin
    x = Image.open(fn).convert(convert_mode)
    x= np.asarray(x)
    if after_open:
        x = after_open(x)
    return x

def stiching_image(img_id, tile_number, N = 36, transform = None, rand = False):
    if tile_number == N:
        fnames = [os.path.join(image_dir, img_id + '_' + str(i) + '.png')
                  for i in range(N)]
    elif tile_number > N:
        idxes = np.random.choice(list(range(tile_number)), N, replace=False)
        fnames = [os.path.join(image_dir, img_id + '_' + str(i) + '.png')
                  for i in idxes]
    else:
        idxes = list(range(tile_number))
        idxes += list(np.random.choice(list(range(tile_number)), N - tile_number, replace=True))
        fnames = [os.path.join(image_dir, img_id + '_' + str(i) + '.png')
                  for i in idxes]

    imgs = []
    for i, fname in enumerate(fnames):
        img = open_image(fname)
        imgs.append({'img': img, 'idx': i})

    if rand: ## random shuffle the order of tiles
        idxes = np.random.choice(list(range(N)), N, replace=False)
    else:
        idxes = list(range(N))

    n_row_tiles = int(np.sqrt(N))

    images = np.zeros((256 * n_row_tiles, 256 * n_row_tiles, 3)).astype(np.uint8)
    for h in range(n_row_tiles):
        for w in range(n_row_tiles):
            i = h * n_row_tiles + w
            if len(imgs) > idxes[i]:
                this_img = imgs[idxes[i]]['img'].astype(np.uint8)
            else:
                this_img = np.ones((256, 256, 3)).astype(np.uint8) * 255
#             this_img = 255 - this_img ## todo: see how this trik plays out
            if transform is not None:
                this_img = transform(image=this_img)['image']
            h1 = h * 256
            w1 = w * 256
            images[h1:h1 + 256, w1:w1 + 256] = this_img

    if transform is not None:
        images = transform(image=images)['image'] 
    return images

In [80]:
for name in names[:5]:
    df = pd.read_csv('./input/csv_pkl_files/4_fold_whole_train.csv')
    idx = df.index[df['image_id'] == name].tolist()
    isup_grade = df.loc[idx[0], 'isup_grade']
    gleason_score = df.loc[idx[0], 'gleason_score']
    tile_number = df.loc[idx[0], 'tile_number']
    img = stiching_image(name, tile_number)
    skimage.io.imsave(f"./input/wrongPreds/label_{isup}_preds_{preds}/{isup}_{gls}_{data_provider}_{name}_patch.png", img)
#     rcParams['figure.figsize'] = 20,10
#     f, axarr = plt.subplots(1,1)
#     axarr.imshow(img)
#     axarr.set_title(f"ID: {image_id}\nSource: {data_provider} ISUP: {isup_grade} Gleason: {gleason_score} Pred: {preds}")
#     f.savefig(f"./input/wrongPreds/label_{isup}_preds_{preds}/{isup}_{gls}_{data_provider}_{img_id}_patch.png")